In [1]:
import numpy as np
import os
from pprint import pprint
import re
from collections import Counter

In [2]:
#utility functions block

def readESymb(loop, name):
    assert os.path.isfile(name)
    res = ''
    prefix = 'Esymb'

    with open(name, 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix + '[' + str(loop) + ']'): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res

def convert(loop, filename):
    dev = re.split(":=|SB\(|\)", re.sub('[,*]', '', readESymb(loop, filename)))[1:-1]
    keys = dev[1::2]
    values = [int(re.sub('[+-]$', t[0] + '1', t)) for t in dev[0::2]]
    out_dict = {}
    for k, v in zip(keys, values):
        out_dict[k] = v
    return out_dict

symbs={}
for L in [1,2,3,4,5,6]:
    if L==6:
        symbs[L]=convert(L,'../data/EZ6_symb_new_norm')
    else: symbs[L]=convert(L,'../data/EZ_symb_new_norm')

In [3]:
from itertools import permutations
alphabet=['a','b','c','d','e','f']
dihedral_table = [list(permutations(alphabet[:3]))[i]+list(permutations(alphabet[3:]))[i] for i in range(len(alphabet))]
dihedral_table = [list(permutations(alphabet[:3]))[i]+list(permutations(alphabet[3:]))[i] for i in range(len(alphabet))]
cycle_table=[dihedral_table[i] for i in [0,3,4]]
flip_table=[dihedral_table[i] for i in [0,1,2,5]]

def get_dihedral_images(word):
    '''
    Get all the dihedral images of a given word.
    ---------
    INPUTS:
    word: str.

    OUTPUTS:
    dihedral_images: list; each item in the list is a word (str); always has six items.
    '''
    word_idx = [alphabet.index(l) for l in [*word]]
    dihedral_images = [''.join([dihedral_table[row][idx] for idx in word_idx]) for row in range(len(alphabet))]
    return dihedral_images

def get_cycle_images(word):
    '''
    Get all the 3-cycle dihedral images of a given word.
    ---------
    INPUTS:
    word: str.

    OUTPUTS:
    cycle_images: list; each item in the list is a word (str); always has six items.
    '''
    word_idx = [alphabet.index(l) for l in [*word]]
    cycle_images = [''.join([cycle_table[row][idx] for idx in word_idx]) for row in range(int(len(alphabet)/2))]
    return cycle_images

def get_flip_images(word):
    '''
    Get all the flip dihedral images of a given word.
    ---------
    INPUTS:
    word: str.

    OUTPUTS:
    cycle_images: list; each item in the list is a word (str); always has six items.
    '''
    my_images = get_dihedral_images(word)
    word_idx = [alphabet.index(l) for l in [*word]]
    cycle_images = [''.join([cycle_table[row][idx] for idx in word_idx]) for row in range(int(len(alphabet)/2))]
    return [im for im in my_images if im not in cycle_images]

In [4]:
def get_partial_dihedrals(word,start,end):
    prefix=word[:start]
    substr=word[start:end]
    suffix=word[end:]
    return [prefix+im+suffix for im in get_dihedral_images(substr)]

def get_partial_cycles(word,start,end):
    prefix=word[:start]
    substr=word[start:end]
    suffix=word[end:]
    return [prefix+im+suffix for im in get_cycle_images(substr)]

def get_partial_flips(word,start,end):
    prefix=word[:start]
    substr=word[start:end]
    suffix=word[end:]
    return [prefix+im+suffix for im in get_flip_images(substr)]

In [5]:
def lookup(word,symb):
    if word in symb: return symb[word]
    else: return 0 

In [6]:
steinmanns={'a':'d', 'b':'e', 'c':'f', 'd':'aef', 'e':'bdf', 'f':'cde'}

def is_triv_zero(word):
    for i,letter in enumerate(word):
        if i == len(word)-1: continue
        else:
            if word[i+1] in steinmanns[letter]: return True
    return False
        

In [7]:
get_partial_dihedrals('abcdef',1,5)

['abcdef', 'acbdff', 'aacedf', 'acaeff', 'aabfdf', 'abafef']

In [8]:
def get_partials(word, start, stop, symb, optype="dihedral"):
    #lookups={k:lookup(k,symb) for k in get_partial_dihedrals(word,start, stop) if k != word}
    if optype=="dihedral":
        nzlookups={k:lookup(k,symb) for k in get_partial_dihedrals(word,start, stop) if not is_triv_zero(k) and k not in get_dihedral_images(word) and k != word}
    elif optype=="cycle":
        nzlookups={k:lookup(k,symb) for k in get_partial_cycles(word,start, stop) if not is_triv_zero(k) and k not in get_dihedral_images(word) and k != word}
    elif optype=="flip":
        nzlookups={k:lookup(k,symb) for k in get_partial_flips(word,start, stop) if not is_triv_zero(k) and k not in get_dihedral_images(word) and k != word}

    
    return nzlookups

In [9]:
symbs[5]

{'caecccbaff': -32,
 'cececaaaee': -24,
 'cceaccbaff': 48,
 'ccecccbaff': -48,
 'abcbccbaff': 48,
 'bbbaacbaff': 60,
 'cabaacbaff': 36,
 'acbaacbaff': 36,
 'aacabcbaff': 32,
 'bbccbcbaff': 468,
 'cdcddbbaff': 48,
 'cabfacbaff': 24,
 'acbfacbaff': 24,
 'bdbfacbaff': 48,
 'cdbfacbaff': 48,
 'caafacbaff': -24,
 'baabbbbaff': 600,
 'cccbbcbaff': -476,
 'bcbaccbaff': 8,
 'cbccccbaff': 560,
 'caeeacbaff': -8,
 'bbcddbbaff': 16,
 'bafffbbaff': 24,
 'bbaccbbaff': -52,
 'afbbbbbaff': 32,
 'cbafbbbaff': -24,
 'aececbbaff': -64,
 'aacbbbbaff': -168,
 'bbcbbbbaff': 480,
 'bbfafbbaff': 32,
 'aaaecbbaff': 40,
 'cdccdbbaff': 8,
 'cbccabbaff': -24,
 'abafbbbaff': 40,
 'abaaccbaff': -68,
 'aeeeecbaff': -576,
 'bcccecbfff': -80,
 'cbcaecbfff': 80,
 'bccaecbfff': 80,
 'aacaccbfff': 1056,
 'bdbcacbfff': -40,
 'afbcccbfff': -480,
 'aeabacbfff': 72,
 'aaeccbbfff': -32,
 'cecccabfff': 104,
 'ceeababfff': 240,
 'cceccbbfff': -96,
 'aacdccbfff': -48,
 'bdcdccbfff': 112,
 'cdcdccbfff': 112,
 'cdbdccbfff': -48,


In [10]:
# extracting value to compare
def magmatch_all(test_dict,coef):
    res=True
    for ele in test_dict:
        if abs(test_dict[ele]) != abs(coef):
            res = False
            break
    return res

def match_all(test_dict,coef):
    res=True
    for ele in test_dict:
        if test_dict[ele] != coef:
            res = False
            break
    return res

def signflip_all(test_dict,coef):
    res=True
    for ele in test_dict:
        if test_dict[ele] != -1*coef:
            res = False
            break
    return res

In [11]:
def allmags_same(word,symb,optype):
    mysames=[]
    for i in range(len(word)):
        for j in range(i+1,len(word)+1):
            partials=get_partials(word,i,j,symb,optype)
            if partials:
                if magmatch_all(partials,symb[word]):mysames.append([i,j])
    return mysames

def allcoefs_same(word,symb,optype):
    mysames=[]
    for i in range(len(word)):
        for j in range(i+1,len(word)+1):
            partials=get_partials(word,i,j,symb,optype)
            if partials:
                if match_all(partials,symb[word]):mysames.append([i,j])
    return mysames

def allcoefs_flip(word,symb,optype):
    mysames=[]
    for i in range(len(word)):
        for j in range(i+1,len(word)+1):
            partials=get_partials(word,i,j,symb,optype)
            if partials:
                if signflip_all(partials,symb[word]):mysames.append([i,j])
    return mysames

In [12]:
def pd_decomps(word,symb,sametype,optype):
    decomps=[]
    if sametype=='mag':
        subs=allmags_same(word,symb,optype)
    elif sametype=='coef':
        subs=allcoefs_same(word,symb,optype)
    elif sametype=='flip':
            subs=allcoefs_flip(word,symb,optype)
    else: raise ValueError
    
    for sub in subs:
        prefix=word[:sub[0]]
        substr=word[sub[0]:sub[1]]
        suffix=word[sub[1]:]
        if prefix != '':
            prefstring=prefix+'+'
        else: prefstring=''
        if suffix != '':
            suffstring='+'+suffix
        else: suffstring=''
        
        if substr == word: continue
        substr=substr.upper()
        #print(sub)
        #print(get_partials(word,sub[0],sub[1],symb))
        decomps.append(f'{prefstring}{substr}{suffstring}')
    return decomps

In [13]:
def ad_flip(key):
     return key.replace('b','c').replace('c','b').replace('f','e').replace('e','f')
    
def binarize(key):
    return key.replace('a','0').replace('b','0').replace('c','0').replace('d','1').replace('e','1').replace('f','1')
    

In [38]:
def make_decomps(loop,sametype,optype):
    mydecomps={}
    for key in symbs[loop]:
        if not abs(symbs[loop][key]) == 24: continue
        #if math.log(abs(symbs[loop][key]), 2).is_integer(): continue
        #if key[0].lower() != 'a': continue
        #if ad_flip(key) in mydecomps: continue
        mydecomps[key]=pd_decomps(key,symbs[loop], sametype=sametype,optype=optype)
        print(key,symbs[loop][key],mydecomps[key])
    #print(len(mydecomps))

In [42]:
make_decomps(3,"mag","dihedral")

afbfbf -24 ['A+fbfbf', 'AFBFB+f', 'a+FBFB+f', 'a+FBFBF', 'afbfb+F']
bfbfbf -24 ['B+fbfbf', 'BFBFB+f', 'b+FBFB+f', 'b+FBFBF', 'bfbfb+F']
bfafaf -24 ['B+fafaf', 'BFAFA+f', 'b+FAFA+f', 'b+FAFAF', 'bfafa+F']
bbfbbf 24 ['BBFBB+f', 'bb+F+bbf', 'bbfbb+F']
bbfbff -24 ['BBFB+ff', 'bb+F+bff', 'bbfb+FF']
bbffbf -24 ['BBFFB+f', 'bb+FF+bf', 'bbffb+F']
aaffaf -24 ['AAFFA+f', 'aa+FF+af', 'aaffa+F']
aafaff -24 ['AAFA+ff', 'aa+F+aff', 'aafa+FF']
aafaaf 24 ['AAFAA+f', 'aa+F+aaf', 'aafaa+F']
afafaf -24 ['A+fafaf', 'AFAFA+f', 'a+FAFA+f', 'a+FAFAF', 'afafa+F']
ceaeaf -24 ['C+eaeaf', 'CEAEA+f', 'c+EAEAF', 'ceaea+F']
afbfbd -24 ['A+fbfbd', 'AFBFB+d', 'a+FBFBD', 'afbfb+D']
bfbfbd -24 ['B+fbfbd', 'BFBFB+d', 'b+FBFB+d', 'b+FBFBD', 'bfbfb+D']
bdbdbf -24 ['B+dbdbf', 'BDBDB+f', 'b+DBDB+f', 'b+DBDBF', 'bdbdb+F']
bbfbbd 24 ['BBFBB+d', 'bb+F+bbd', 'bbfbb+D']
bbffbd -24 ['BBFFB+d', 'bb+FF+bd', 'bbffb+D']
bbfbdd 24 ['BBFB+dd', 'bb+F+bdd', 'bbfb+DD']
bbddbf -24 ['BBDDB+f', 'bb+DD+bf', 'bbddb+F']
bbdbbf 24 ['BBDBB+f', 'b

In [15]:
import math


pprint(symbs[1])
pprint(symbs[2])
pprint({k:v for k,v in symbs[3].items() if not math.log(abs(v), 2).is_integer()})
#pprint({k:v for k,v in symbs[3].items() if v == 4})

{'ae': -2, 'af': -2, 'bd': -2, 'bf': -2, 'cd': -2, 'ce': -2}
{'aaae': 16,
 'aaaf': 16,
 'aeee': 8,
 'afff': 8,
 'bbbd': 16,
 'bbbf': 16,
 'bddd': 8,
 'bfff': 8,
 'cccd': 16,
 'ccce': 16,
 'cddd': 8,
 'ceee': 8}
{'aaaaae': -384,
 'aaaaaf': -384,
 'aaaeae': 28,
 'aaaeaf': 28,
 'aaaeee': -56,
 'aaafae': 28,
 'aaafaf': 28,
 'aaafff': -56,
 'aaeaae': 24,
 'aaeaaf': 24,
 'aaeaee': -24,
 'aaeaff': 24,
 'aaeeae': -24,
 'aaeeaf': -24,
 'aafaae': 24,
 'aafaaf': 24,
 'aafaee': 24,
 'aafaff': -24,
 'aaffae': -24,
 'aaffaf': -24,
 'aeaeae': -24,
 'aeaeaf': -24,
 'aececd': -24,
 'aecece': -24,
 'aeeeee': -96,
 'afafae': -24,
 'afafaf': -24,
 'afbfbd': -24,
 'afbfbf': -24,
 'afffff': -96,
 'bbbbbd': -384,
 'bbbbbf': -384,
 'bbbdbd': 28,
 'bbbdbf': 28,
 'bbbddd': -56,
 'bbbfbd': 28,
 'bbbfbf': 28,
 'bbbfff': -56,
 'bbdbbd': 24,
 'bbdbbf': 24,
 'bbdbdd': -24,
 'bbdbff': 24,
 'bbddbd': -24,
 'bbddbf': -24,
 'bbfbbd': 24,
 'bbfbbf': 24,
 'bbfbdd': 24,
 'bbfbff': -24,
 'bbffbd': -24,
 'bbffbf': -24,
 'bdb

In [16]:
make_decomps(3,"flip","dihedral")

accbff -4 ['ACCB+ff', 'a+CCBF+f', 'ac+CBF+f', 'acc+B+ff', 'acc+BF+f', 'accb+FF']
acbfff 8 ['a+CBF+ff', 'a+CBFF+f', 'ac+B+fff', 'ac+BF+ff', 'ac+BFF+f']
acbfbf -4 ['ac+B+fbf']
acabff 4 ['ACAB+ff', 'aca+B+ff', 'aca+BF+f', 'acab+FF']
aacbff 4 ['AACB+ff', 'aa+CBF+f', 'aac+B+ff', 'aac+BF+f', 'aacb+FF']
acafbf 4 ['ac+A+fbf']
aaffbf 8 ['a+A+ffbf', 'a+AF+fbf']
aafbff 8 ['AAFB+ff', 'a+A+fbff', 'a+AFBF+f', 'aafb+FF']
aafbbf -8 ['a+A+fbbf']
afbbff -16 ['AFBB+ff', 'afbb+FF']
abfbff -8 ['ABFB+ff', 'a+B+fbff', 'abfb+FF']
abfbbf 8 ['a+B+fbbf']
abfaff -8 ['ABFA+ff', 'a+BFAF+f', 'abfa+FF']
abfaaf 8 []
aabfff -8 []
aabfaf 4 []
abffbf -8 ['a+B+ffbf', 'a+BF+fbf']
afbfff 16 []
abffaf -8 []
aabbff -4 ['AABB+ff', 'a+ABBF+f', 'aab+B+ff', 'aab+BF+f', 'aabb+FF']
aabaff 4 ['AABA+ff', 'a+ABAF+f', 'aab+A+ff', 'aab+AF+f', 'aaba+FF']
afbfbf -24 []
abbfaf -4 ['ab+B+faf', 'abbf+A+f']
abbfbf 4 ['ab+B+fbf', 'abbf+B+f']
abbbff 4 ['ABBB+ff', 'abb+B+ff', 'abb+BF+f', 'abbb+FF']
abbaff -4 ['ABBA+ff', 'abb+A+ff', 'abb+AF+f', '

In [17]:
#Get count of singlet, doublet, etc. orbits at each loop
for i in [1,2,3,4,5,6,7,8]:
    orbits=[myorbs["all"][L].get(i,None) for L in [1,2,3,4,5,6]]
    absorbits=[myorbs["abs"][L].get(i,None) for L in [1,2,3,4,5,6]]
    print(f"Orbits of size {i}: {orbits}")
    print(f"Mag orbits of size {i}: {absorbits}")
    print("\n")

NameError: name 'myorbs' is not defined

In [ ]:
#We note something interesting: at L>5 there are many small orbits (singlets, doublets, etc.):
#all orbit sizes [1... N_break] are represented: counting up by one, not integer multiples of each other!
#(it's kind of weird to me that we have a lot of orbits of size 10, 11, 12, and 13, for example).
#Let's find the break_number.

def get_breakpoint(L, mydictlist):
    bp=0
    i=1
    while i < 10000:
        if i in mydictlist[L]:
            i+=1
            continue
        else:
            return i

#Last orbit size before breakpoint (do it this way to make L=1 make sense)             
bp_all = [get_breakpoint(L, myorbs["all"])-1 for L in [1,2,3,4,5,6]]
bp_mag = [get_breakpoint(L, myorbs["abs"])-1 for L in [1,2,3,4,5,6]]
bp_split = [get_breakpoint(L, myorbs["split"])-1 for L in [1,2,3,4,5,6]]
bp_bal = [get_breakpoint(L, myorbs["balanced"])-1 for L in [1,2,3,4,5,6]]
bp_ubal = [get_breakpoint(L, myorbs["unbalanced"])-1 for L in [1,2,3,4,5,6]]

print(bp_all)
print(bp_mag)
print(bp_split)
print(bp_bal)
print(bp_ubal)

In [10]:
#Let's go back to look at coefs again: some coefs only show up as one sign, and others don't.
#This might be a natural way to split up the group-theory counting even more

In [11]:
#Let's check to make sure things make sense at L=4:
#We see that at L=4, 40 is evenly split, but 80 is not. 
print(sort_orbits(4,"study")[0])

{112: [6, 8], 288: [10, 2], 144: [3, 7], 96: [37, 29], 48: [77, 77], 128: [21, 19], 256: [10, 2], 8: [188, 192], 32: [168, 172], 16: [221, 229], 40: [42, 42], 80: [23, 33], 480: [3, 1], 64: [65, 58], 352: [4, 2], 384: [11, 19], 240: [10, 2], 448: [3, 1], 160: [8, 4], 72: [2, 2], 264: [2, 2], 192: [4, 7], 304: [1, 5]}


In [12]:
#At L=6, note that split size tends towards being a small fraction of total orbit size
#ex: 704: [6037, 6094]. At L=6, abs(split size) is at most 129
print(sort_orbits(6,"study")[0])

{704: [6037, 6094], 1536: [3187, 3180], 1960: [70, 70], 352: [3502, 3547], 768: [9447, 9452], 1024: [6467, 6539], 256: [14102, 13999], 160: [7968, 8003], 592: [1560, 1559], 1280: [3044, 3165], 168: [546, 546], 736: [2645, 2586], 656: [1441, 1433], 456: [252, 252], 920: [63, 63], 1152: [3806, 3817], 528: [757, 773], 128: [15318, 15258], 544: [2851, 2837], 904: [432, 432], 688: [1409, 1409], 1344: [1863, 1855], 24: [665, 661], 2528: [332, 339], 2688: [666, 656], 11968: [39, 31], 2016: [493, 488], 1104: [865, 865], 416: [5709, 5668], 192: [8065, 8092], 8768: [136, 159], 288: [4708, 4673], 608: [4233, 4221], 824: [481, 481], 832: [4576, 4581], 1408: [2995, 2976], 512: [13143, 13093], 976: [527, 535], 480: [3321, 3354], 96: [4822, 4858], 56: [1934, 1938], 280: [209, 209], 2112: [765, 727], 1600: [1002, 958], 3776: [218, 205], 624: [1993, 1977], 640: [7083, 7034], 960: [3610, 3581], 752: [1592, 1610], 1184: [1495, 1537], 384: [14151, 14071], 32: [5491, 5569], 560: [2613, 2617], 368: [956, 94

In [44]:
splitchecker,uneven_diffs,even_roots ={},{},{}
for loop in [1,2,3,4,5,6]:
    splitchecker[loop],even_roots[loop],uneven_diffs[loop]=sort_orbits(loop,"study")    


#Let's look at unevenly split coefs and get:
print("Size of uneven set")
print([len(uneven_diffs[L].keys()) for L in [2,3,4,5,6]])
print("Largest coef with an uneven split: L > 2")
print([max(uneven_diffs[L].keys()) for L in [3,4,5,6]])
print("Coef with largest uneven split: L > 2") #These are all in A001815: almost binomial(n,2) * 2^(n-1)
print([max(uneven_diffs[L], key=lambda y: abs(uneven_diffs[L][y])) for L in [3,4,5,6]])
print("Split size of coef with largest uneven split: L > 2")
print([uneven_diffs[L][max(uneven_diffs[L], key=lambda y: abs(uneven_diffs[L][y]))] for L in [3,4,5,6]])

#Let's look at evenly split coefs and get:
print("Size of even set")
print([len(even_roots[L].keys()) for L in [2,3,4,5,6]])
print("Largest coef with an even-split orbit: L > 3")
print([max(even_roots[L].keys()) for L in [4,5,6]])
print("Coef with largest even-split orbit: L > 3")
print([max(even_roots[L], key=lambda y: abs(even_roots[L][y])) for L in [4,5,6]])

Size of uneven set
[0, 4, 19, 170, 1297]
Largest coef with an uneven split: L > 2
[24, 480, 23040, 1720320]
Coef with largest uneven split: L > 2
[16, 80, 96, 1920]
Split size of coef with largest uneven split: L > 2
[8, -10, -80, 129]
Size of even set
[0, 0, 4, 100, 766]
Largest coef with an even-split orbit: L > 3
[264, 16256, 891120]
Coef with largest even-split orbit: L > 3
[48, 56, 688]


In [46]:
splitchecker[6]

{704: [6037, 6094],
 1536: [3187, 3180],
 1960: [70, 70],
 352: [3502, 3547],
 768: [9447, 9452],
 1024: [6467, 6539],
 256: [14102, 13999],
 160: [7968, 8003],
 592: [1560, 1559],
 1280: [3044, 3165],
 168: [546, 546],
 736: [2645, 2586],
 656: [1441, 1433],
 456: [252, 252],
 920: [63, 63],
 1152: [3806, 3817],
 528: [757, 773],
 128: [15318, 15258],
 544: [2851, 2837],
 904: [432, 432],
 688: [1409, 1409],
 1344: [1863, 1855],
 24: [665, 661],
 2528: [332, 339],
 2688: [666, 656],
 11968: [39, 31],
 2016: [493, 488],
 1104: [865, 865],
 416: [5709, 5668],
 192: [8065, 8092],
 8768: [136, 159],
 288: [4708, 4673],
 608: [4233, 4221],
 824: [481, 481],
 832: [4576, 4581],
 1408: [2995, 2976],
 512: [13143, 13093],
 976: [527, 535],
 480: [3321, 3354],
 96: [4822, 4858],
 56: [1934, 1938],
 280: [209, 209],
 2112: [765, 727],
 1600: [1002, 958],
 3776: [218, 205],
 624: [1993, 1977],
 640: [7083, 7034],
 960: [3610, 3581],
 752: [1592, 1610],
 1184: [1495, 1537],
 384: [14151, 14071],


In [14]:
"""
#Let's count per loop!
#Get singlets, doublets, etc. in each subspace 
#for i in [1,2,3,4]:
    print(f"Orbits of size {i}:")
    print("Pure coefs")
    print([orb.get(i,None) for orb in pure_orbits])
    print("Split coefs")    
    print([orb.get(i,None) for orb in split_orbits])
    print("Pure coefs, positive")
    print([orb.get(i,None) for orb in pure_pos_orbits])
    print("Pure coefs, negative")    
    print([orb.get(i,None) for orb in pure_neg_orbits])    
    print("Even splits")
    print([orb.get(i,None) for orb in even_orbits])
    print("Uneven Splits")    
    print([orb.get(i,None) for orb in uneven_orbits])
    print("\n")
"""

'\n#Let\'s count per loop!\n#Get singlets, doublets, etc. in each subspace \n#for i in [1,2,3,4]:\n    print(f"Orbits of size {i}:")\n    print("Pure coefs")\n    print([orb.get(i,None) for orb in pure_orbits])\n    print("Split coefs")    \n    print([orb.get(i,None) for orb in split_orbits])\n    print("Pure coefs, positive")\n    print([orb.get(i,None) for orb in pure_pos_orbits])\n    print("Pure coefs, negative")    \n    print([orb.get(i,None) for orb in pure_neg_orbits])    \n    print("Even splits")\n    print([orb.get(i,None) for orb in even_orbits])\n    print("Uneven Splits")    \n    print([orb.get(i,None) for orb in uneven_orbits])\n    print("\n")\n'

In [15]:
for orbtype in orbtypes:
    print("\n")
    print(f"For orbit type {orbtype}")
    print_orbstats(mylens[orbtype], myorbs[orbtype], orbtype)



For orbit type all
Number of distinct orbits
[1, 2, 12, 59, 648, 4875]
Size of largest orbit:
[1, 1, 21, 229, 1897, 15318]
How many orbits of largest size:
[1, 2, 1, 1, 1, 1]
Most common orbit size:
[1, 1, 1, 2, 2, 2]
How many orbits of most common size:
[1, 2, 3, 12, 103, 907]
How many singlets:
[1, 2, 3, 7, 76, 580]
How many different orbit sizes:
[1, 1, 9, 26, 138, 543]
Total dimensionality of space:
[1, 2, 106, 1868, 43980, 819411]


For orbit type abs
Number of distinct orbits
[1, 2, 8, 36, 378, 2812]
Size of largest orbit:
[1, 1, 40, 450, 3767, 30576]
How many orbits of largest size:
[1, 2, 1, 1, 1, 1]
Most common orbit size:
[1, 1, 1, 4, 2, 2]
How many orbits of most common size:
[1, 2, 3, 8, 70, 529]
How many singlets:
[1, 2, 3, 4, 21, 147]
How many different orbit sizes:
[1, 1, 6, 18, 117, 449]
Total dimensionality of space:
[1, 2, 106, 1868, 43980, 819411]


For orbit type pos
Number of distinct orbits
[0, 2, 5, 30, 318, 2469]
Size of largest orbit:
[0, 1, 21, 221, 1870, 15

In [38]:
orbit_dict=count_orbits(4,"pure")
pprint(orbit_dict)
pprint(get_orbits(4,"pure",1920, orbit_dict))

{-1248: 2,
 -1216: 2,
 -1152: 4,
 -768: 4,
 -224: 2,
 -88: 4,
 272: 4,
 320: 6,
 544: 2,
 576: 1,
 960: 1,
 1920: 1,
 15360: 1}
['bddddddd', 'ceeeeeee', 'bfffffff', 'afffffff', 'aeeeeeee', 'cddddddd']


In [17]:
#so, what can we say?
"""
-
-If we exclude "pure" coeffs that are all + or all -, the asymmetry between + and - is consistently very small
-The "pure" coeffs appear to be the b-rays and d-rays!
-no obvious recurrences in OEIS- but maybe worth trying things like binomial/Mobius transform 
-Need to look more closely at sign asymmetry
"""

'\n-\n-If we exclude "pure" coeffs that are all + or all -, the asymmetry between + and - is consistently very small\n-no obvious recurrences in OEIS- but maybe worth trying things like binomial/Mobius transform\n-Need to look at individual coefs, but hard to see structure unless we divide out common factors:\n    maybe band structure?\n'